In [1]:
# Generating video resume content with enhanced features

import spacy
from PyPDF2 import PdfReader
import os
from openai import OpenAI
from dotenv import load_dotenv
import json

# Load environment variables from .env file
load_dotenv()

# Initialize OpenAI client
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY not found in environment variables. Please set it in your .env file.")

client = OpenAI(api_key=api_key)

# Load the English NLP model
try:
    nlp = spacy.load("en_core_web_sm")
except OSError:
    print("Warning: spaCy model not found. Please run: python -m spacy download en_core_web_sm")
    nlp = None

# Taking role and company as input
print("Enter the job role you're applying for:")
role = input().strip()

print("Enter the company name:")
company = input().strip()

# Open the resume file and read the text
try:
    with open('Main.pdf', 'rb') as f:
        pdf_reader = PdfReader(f)
        resume_text = ''
        for page_number in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_number]
            page_text = page.extract_text()
            resume_text += page_text
    print(f"\n✓ Resume loaded successfully ({len(pdf_reader.pages)} pages)")
except FileNotFoundError:
    print("Warning: Main.pdf not found. Using placeholder text.")
    resume_text = "Software Engineer with 5 years of experience in Python, JavaScript, and cloud technologies."

# Get keywords relevant for the role
print(f"\n🔍 Analyzing keywords for {role} position...")
keywords_response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "system",
            "content": "You are a career advisor helping to identify relevant keywords for job applications."
        },
        {
            "role": "user",
            "content": f"Can you provide a comprehensive list of keywords (skills, qualifications, experiences) that would be relevant for a {role} position in a resume? Format as a clean list, one item per line."
        }
    ],
    temperature=0.7,
    max_tokens=500
)

key_phrases = keywords_response.choices[0].message.content
print(f"\nRelevant Keywords:\n{key_phrases}\n")

# Generate video resume content
print(f"🎬 Generating personalized video resume content for {company}...")
video_content_response = client.chat.completions.create(
    model="gpt-4.1",
    messages=[
        {
            "role": "system",
            "content": "You are a professional resume writer creating compelling video resume scripts."
        },
        {
            "role": "user",
            "content": f"""Generate a professional video resume script (2-3 minutes speaking time) based on the following resume:

{resume_text[:2000]}

Target company: {company}
Target role: {role}
Relevant keywords to emphasize: {key_phrases[:500]}

The script should:
1. Start with a strong introduction
2. Highlight relevant experience and skills
3. Show enthusiasm for the role and company
4. End with a compelling call to action
5. Be natural and conversational (not robotic)

Format the output as a clear script with timing cues."""
        }
    ],
    temperature=0.8,
    max_tokens=1000
)

video_script = video_content_response.choices[0].message.content
print("\n" + "="*60)
print("VIDEO RESUME SCRIPT")
print("="*60)
print(video_script)

# Generate certificate recommendations
print("\n\n📜 Certificate Analysis:")
print("Enter your certificates (comma-separated):")
cert_input = input().strip()
if cert_input:
    certs = [c.strip() for c in cert_input.split(",")]
    relevant_certs = []
    for cert in certs:
        if cert.lower() in key_phrases.lower():
            relevant_certs.append(cert)
    
    if relevant_certs:
        print(f"\n✓ Relevant certificates for {role}:")
        for cert in relevant_certs:
            print(f"  • {cert}")
    else:
        print(f"\n⚠ No certificates directly match the keywords, but all certifications add value to your profile.")

Enter the job role you're applying for:
Enter the company name:

🔍 Analyzing keywords for software engineer position...

Relevant Keywords:
- Software development
- Programming languages (e.g., Java, Python, C++)
- Software design
- Algorithms
- Data structures
- Object-oriented design
- Database management
- Web development
- Mobile development
- Agile methodology
- Software testing
- Debugging
- Problem-solving
- Team collaboration
- Version control (e.g., Git)
- Continuous integration/continuous deployment (CI/CD)
- Cloud computing
- Front-end development
- Back-end development
- Full stack development
- User interface/user experience (UI/UX) design
- Software documentation
- Project management
- Technical writing
- Troubleshooting
- Communication skills
- Bachelor's degree in Computer Science or related field

🎬 Generating personalized video resume content for microsoft...

VIDEO RESUME SCRIPT
**[0:00 – 0:20 | Introduction]**

Hi, I’m [Your Name], and I’m a passionate software engi

In [ ]:
# Enhanced Certificate Recommendation System

import os
from openai import OpenAI
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY not found in environment variables. Please set it in your .env file.")

client = OpenAI(api_key=api_key)

print("Enter the job role:")
role = input().strip()

# Get comprehensive keywords for the role
print(f"\n🔍 Analyzing role requirements for {role}...")
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "system",
            "content": "You are a career advisor specializing in certifications and professional development."
        },
        {
            "role": "user",
            "content": f"""For a {role} position, provide:
1. Essential skills and technologies
2. Recommended certifications
3. Industry-standard qualifications

Format as a clear list with categories."""
        }
    ],
    temperature=0.7,
    max_tokens=500
)

key = response.choices[0].message.content
print(f"\n📋 Role Analysis:\n{key}\n")

# Get user's certificates
print("Enter your certificates (comma-separated):")
cert_input = input().strip()
certs = [c.strip() for c in cert_input.split(",")] if cert_input else []

# Analyze which certificates are relevant
if certs:
    print(f"\n📜 Analyzing {len(certs)} certificate(s)...")
    relevant_certs = []
    partially_relevant = []
    
    for cert in certs:
        cert_lower = cert.lower()
        key_lower = key.lower()
        
        # Check for direct matches or partial matches
        if any(word in key_lower for word in cert_lower.split() if len(word) > 3):
            relevant_certs.append(cert)
        elif any(word in cert_lower for word in key_lower.split()[:20] if len(word) > 3):
            partially_relevant.append(cert)
    
    print("\n" + "="*60)
    if relevant_certs:
        print("✓ Highly Relevant Certificates:")
        for cert in relevant_certs:
            print(f"  • {cert}")
    
    if partially_relevant:
        print("\n✓ Partially Relevant Certificates:")
        for cert in partially_relevant:
            print(f"  • {cert}")
    
    if not relevant_certs and not partially_relevant:
        print("ℹ All certificates add value to your profile, even if not directly matching keywords.")
        print("Consider highlighting transferable skills from these certifications.")
else:
    print("No certificates provided.")


Machine learning
Python, c++, Autocad
Python


In [ ]:
# Professional Cold Email Generator

import os
from openai import OpenAI
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY not found in environment variables. Please set it in your .env file.")

client = OpenAI(api_key=api_key)

print("Enter the job role you're applying for:")
role = input().strip()

print("Enter the employer/recruiter name:")
emp = input().strip()

print("Enter your name:")
name = input().strip()

print("Enter company name (optional, press Enter to skip):")
company = input().strip()

# Generate professional cold email
print(f"\n✉️ Generating professional cold email...")
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "system",
            "content": "You are a professional career coach specializing in networking and job applications. Create compelling, personalized cold emails."
        },
        {
            "role": "user",
            "content": f"""Generate a professional cold email with the following details:
- From: {name} (student/job seeker)
- To: {emp}
- Purpose: Expressing interest in {role} position
- Company: {company if company else 'the company'}

Requirements:
1. Professional but warm tone
2. Clear subject line suggestion
3. Brief introduction
4. Express genuine interest in the role
5. Highlight relevant skills/experience briefly
6. Request for opportunity to connect or discuss
7. Professional closing

Keep it concise (150-200 words), authentic, and avoid being pushy."""
        }
    ],
    temperature=0.8,
    max_tokens=500
)

email_content = response.choices[0].message.content
print("\n" + "="*60)
print("COLD EMAIL")
print("="*60)
print(email_content)
print("\n💡 Tip: Personalize this email further by mentioning specific company achievements or recent news.")

Software Developer
Srinivas
Mani


Subject: Hi Srinivas, I'm Mani, a Software Developer

Hi Srinivas,

My name is Mani and I'm a student looking to break into the software development industry. I heard through a mutual friend that you're a big name in the software development industry, and I wanted to reach out to you for advice and guidance.

I'm sure you have a busy schedule, but I would really appreciate it if you could take a few minutes out of your day to chat with me about my career goals and how I can best position myself to land a software development role.

I look forward to hearing from you soon.

Sincerely,
Mani


In [ ]:
# LinkedIn/Social Media Cold Message Generator

import os
from openai import OpenAI
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY not found in environment variables. Please set it in your .env file.")

client = OpenAI(api_key=api_key)

print("Enter the job role:")
role = input().strip()

print("Enter the contact person's name:")
emp = input().strip()

print("Enter your name:")
name = input().strip()

print("Enter platform (LinkedIn/Twitter/Email - default: LinkedIn):")
platform = input().strip() or "LinkedIn"

# Generate social media cold message
print(f"\n💬 Generating {platform} message...")
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "system",
            "content": "You are a networking expert creating engaging social media messages for professional connections."
        },
        {
            "role": "user",
            "content": f"""Generate a professional but friendly {platform} message with:
- From: {name}
- To: {emp}
- Purpose: Expressing interest in {role} position and requesting connection/advice

Requirements:
1. Platform-appropriate tone ({platform} is more casual than email but still professional)
2. Brief and engaging (100-150 words)
3. Personal touch
4. Clear value proposition
5. Call to action (connection request or brief chat)

Make it authentic and avoid being salesy."""
        }
    ],
    temperature=0.8,
    max_tokens=400
)

message = response.choices[0].message.content
print("\n" + "="*60)
print(f"{platform.upper()} MESSAGE")
print("="*60)
print(message)
print("\n💡 Tip: Always personalize the first line based on their profile or recent posts.")

software developer
srinivas
mani


Dear Srinivas,

My name is Mani and I am currently a software developer. I wanted to reach out to you and ask if you would be interested in a potential job opportunity in software development. I believe that you have the skills necessary to excel in this role, and I would love to speak further about this opportunity.

Thanks for your time and I look forward to hearing from you soon.

Sincerely,
Mani


In [ ]:
# Enhanced Resume Analysis with AI-Powered Insights

from PyPDF2 import PdfReader
import spacy
import os
from openai import OpenAI
from dotenv import load_dotenv
import re

# Load environment variables
load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY not found in environment variables. Please set it in your .env file.")

client = OpenAI(api_key=api_key)

# Load the English NLP model
try:
    nlp = spacy.load("en_core_web_sm")
    print("✓ NLP model loaded")
except OSError:
    print("⚠ Warning: spaCy model not found. Some features may be limited.")
    nlp = None

# Get job specifications
print("Enter job description or specifications:")
job_specifications = input().strip()

if not job_specifications:
    job_specifications = "We are looking for a candidate with at least 3 years of experience in software development. The candidate should have experience with Python, Django, and React. Good communication skills and ability to work in a team is a plus."
    print(f"Using default job spec: {job_specifications}")

# Open the resume file and read the text
try:
    with open('Main.pdf', 'rb') as f:
        pdf_reader = PdfReader(f)
        resume_text = ''
        for page_number in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_number]
            page_text = page.extract_text()
            resume_text += page_text
    print(f"✓ Resume loaded ({len(pdf_reader.pages)} pages)")
except FileNotFoundError:
    print("⚠ Main.pdf not found. Using sample text.")
    resume_text = "Software Engineer with 5 years of experience. Proficient in Python, Django, React, JavaScript. Strong communication skills and team collaboration experience."

# Extract keywords from job specifications using AI
print("\n🔍 Extracting keywords from job description...")
keywords_response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "system",
            "content": "You are an ATS (Applicant Tracking System) expert analyzing job descriptions."
        },
        {
            "role": "user",
            "content": f"""Analyze this job description and extract:
1. Required skills (technical and soft skills)
2. Required experience/qualifications
3. Key responsibilities
4. Preferred qualifications

Job Description:
{job_specifications}

Return a structured list of keywords with importance levels."""
        }
    ],
    temperature=0.3,
    max_tokens=500
)

extracted_keywords = keywords_response.choices[0].message.content
print(f"\n📋 Extracted Keywords:\n{extracted_keywords}\n")

# Enhanced resume analysis function
def analyze_resume(resume_text, job_spec):
    score = 0
    matched_keywords = []
    missing_keywords = []
    
    if nlp:
        resume_doc = nlp(resume_text.lower())
        job_doc = nlp(job_spec.lower())
        
        # Extract important terms from job spec
        job_keywords = [token.text for token in job_doc if token.pos_ in ['NOUN', 'PROPN'] and len(token.text) > 3]
        resume_tokens = [token.text for token in resume_doc]
        
        # Calculate matches
        for keyword in set(job_keywords):
            if keyword in resume_tokens:
                matched_keywords.append(keyword)
                score += 1
            else:
                missing_keywords.append(keyword)
    else:
        # Fallback: simple text matching
        resume_lower = resume_text.lower()
        job_lower = job_spec.lower()
        
        # Extract meaningful words
        job_words = set(re.findall(r'\b[a-z]{4,}\b', job_lower))
        resume_words = set(re.findall(r'\b[a-z]{4,}\b', resume_lower))
        
        matched_keywords = list(job_words.intersection(resume_words))
        missing_keywords = list(job_words - resume_words)
        score = len(matched_keywords)
    
    return score, matched_keywords, missing_keywords

# Analyze resume
print("📊 Analyzing resume against job requirements...")
resume_score, matched, missing = analyze_resume(resume_text, job_specifications)

# Get AI-powered feedback
print("\n🤖 Getting AI feedback...")
feedback_response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "system",
            "content": "You are a career advisor providing constructive feedback on resume optimization."
        },
        {
            "role": "user",
            "content": f"""Resume Analysis Results:
- Match Score: {resume_score} keywords matched
- Matched Keywords: {', '.join(matched[:10])}
- Missing Keywords: {', '.join(missing[:10])}

Job Requirements:
{job_specifications[:500]}

Provide:
1. Overall assessment (0-100 score)
2. Strengths
3. Areas for improvement
4. Specific recommendations to improve the resume"""
        }
    ],
    temperature=0.7,
    max_tokens=600
)

feedback = feedback_response.choices[0].message.content

# Display results
print("\n" + "="*60)
print("RESUME ANALYSIS RESULTS")
print("="*60)
print(f"\n📈 Match Score: {resume_score} keywords found")
print(f"\n✓ Matched Keywords ({len(matched)}):")
for kw in matched[:15]:
    print(f"  • {kw}")

if missing:
    print(f"\n⚠ Missing Keywords ({len(missing)}):")
    for kw in missing[:15]:
        print(f"  • {kw}")

print("\n" + "="*60)
print("AI FEEDBACK & RECOMMENDATIONS")
print("="*60)
print(feedback)

Resume score: 2.1


In [ ]:
# Resume Enhancement Suggestions Generator

import os
from openai import OpenAI
from dotenv import load_dotenv
from PyPDF2 import PdfReader

# Load environment variables
load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY not found in environment variables. Please set it in your .env file.")

client = OpenAI(api_key=api_key)

# Load resume
try:
    with open('Main.pdf', 'rb') as f:
        pdf_reader = PdfReader(f)
        resume_text = ''
        for page_number in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_number]
            resume_text += page.extract_text()
    print("✓ Resume loaded")
except FileNotFoundError:
    print("⚠ Main.pdf not found. Please ensure the file exists.")
    resume_text = ""

if resume_text:
    print("\nEnter target job role:")
    target_role = input().strip()
    
    print("\n🎯 Generating enhancement suggestions...")
    enhancement_response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": "You are a professional resume writer and career coach with expertise in ATS optimization."
            },
            {
                "role": "user",
                "content": f"""Analyze this resume and provide specific enhancement suggestions for a {target_role} position:

{resume_text[:3000]}

Provide:
1. Summary/Objective improvement
2. Skills section optimization
3. Experience bullet point enhancements (use action verbs, quantify achievements)
4. Missing sections to add
5. Keywords to incorporate
6. Formatting suggestions

Be specific and actionable."""
            }
        ],
        temperature=0.7,
        max_tokens=800
    )
    
    suggestions = enhancement_response.choices[0].message.content
    print("\n" + "="*60)
    print("RESUME ENHANCEMENT SUGGESTIONS")
    print("="*60)
    print(suggestions)
else:
    print("Please add Main.pdf to use this feature.")
